In [1]:
import pandas as pd
import re
import multiprocessing

In [2]:
# Read the ground truth files for SOTAB
# cpa_train_gt = pd.read_csv('data/CPA/CPA_training_gt.csv')
# cpa_val_gt = pd.read_csv('data/CPA/CPA_validation_gt.csv')
# cpa_test_gt = pd.read_csv('data/CPA/CPA_test_gt.csv')

In [2]:
# Read the ground truth files for SOTABv2
cpa_train_gt = pd.read_csv('../SOTAB-v2/CPA/sotab_v2_cpa_training_set.csv')
cpa_val_gt = pd.read_csv('../SOTAB-v2/CPA/sotab_v2_cpa_validation_set.csv')
cpa_test_gt = pd.read_csv('../SOTAB-v2/CPA/sotab_v2_cpa_test_set.csv')

In [3]:
gt = {'train':{}, 'val':{}, 'test':{}}
for index, row in cpa_train_gt.iterrows():
    
    if row['table_name'] not in gt['train']:
        gt['train'][row['table_name']] = {}
        
    gt['train'][row['table_name']][row['column_index']] = row['label']
val = {}
for index, row in cpa_val_gt.iterrows():
    if row['table_name'] not in gt['val']:
        gt['val'][row['table_name']] = {} 
    gt['val'][row['table_name']][row['column_index']] = row['label']
    
test = {}
for index, row in cpa_test_gt.iterrows():
    if row['table_name'] not in gt['test']:
        gt['test'][row['table_name']] = {}
    gt['test'][row['table_name']][row['column_index']] = row['label']

In [4]:
type_labels = list(cpa_test_gt["label"].unique())
print(len(type_labels))

In [6]:
cpa_train_cols = (cpa_train_gt['table_name'] + '|' + cpa_train_gt['column_index'].map(str) + '|' + cpa_train_gt['label']).tolist()
cpa_val_cols = (cpa_val_gt['table_name'] + '|' + cpa_val_gt['column_index'].map(str) + '|' + cpa_val_gt['label']).tolist()
cpa_test_cols = (cpa_test_gt['table_name'] + '|' + cpa_test_gt['column_index'].map(str) + '|' + cpa_test_gt['label']).tolist() #+ '|' + cpa_test_gt['selection_type']

In [7]:
#Simple Preprocessing

def clean_text(text):
    
    if(isinstance(text, dict)):
        text = ' '.join([ clean_text(v) for k, v in text.items()] )
    elif(isinstance(text, list)):
        text = map(clean_text, text)
        text = ' '.join(text)
        
    if pd.isnull(text):
        return ''
    
    #Remove excess whitespaces
    text = re.sub(' +', ' ', str(text)).strip()
    
    return text


In [8]:
# Prepare format of input datasets for Doduo: table_id, [labels], data, label_ids
def get_table_column(column):
#     try:
    file_name, column_index, label = column.split('|') #, sel

    #By column
    if file_name in cpa_train_gt['table_name'].tolist():
        path = '../SOTAB-v2/CPA/Train/'+file_name
    elif file_name in cpa_val_gt['table_name'].tolist():
        path = '../SOTAB-v2/CPA/Validation/'+file_name
    else:
        path = '../SOTAB-v2/CPA/Test/'+file_name

    df = pd.read_json(path, compression='gzip', lines=True)

    y = [0] * len(type_labels)
    y[type_labels.index(label)] = 1

    return [
        file_name, #table_id
        column_index, #column_id
        [label], #[labels]
        clean_text(df.iloc[:, int(column_index)].tolist()), #data
        y, #label_ids
        column_index
    ]

In [9]:
pool = multiprocessing.Pool(processes=20)
train_result = pool.map(get_table_column, cpa_train_cols)
val_result = pool.map(get_table_column, cpa_val_cols)
val_result = [val for val in val_result if val ]
test_result = pool.map(get_table_column, cpa_test_cols)
pool.close()
pool.join()

In [12]:
def get_main_column(file_name):
    main_col_index = 0
    
    if file_name in cpa_train_gt['table_name'].tolist():
        path = 'SOTAB-v2/CPA/Train/'+file_name
    elif file_name in cpa_val_gt['table_name'].tolist():
        path = 'SOTAB-v2/CPA/Validation/'+file_name
    else:
        path = 'SOTAB-v2/CPA/Test/'+file_name
    
    #Open table
    df = pd.read_json(path, compression='gzip', lines=True)
    
    y = [0] * len(type_labels)

    return [
        file_name, #table_id
        main_col_index, #main column index
        [], #[labels]
        clean_text(df[main_col_index].tolist()), #data
        y, #label_ids
        main_col_index
    ] 

In [13]:
pool = multiprocessing.Pool(processes=20)
train_main_cols = pool.map(get_main_column, cpa_train_gt['table_name'].unique())
val_main_cols = pool.map(get_main_column, cpa_val_gt['table_name'].unique())
test_main_cols = pool.map(get_main_column, cpa_test_gt['table_name'].unique())
pool.close()
pool.join()

In [14]:
cpa = {}
cpa['train'] = pd.DataFrame(train_result + train_main_cols, columns=['table_id', 'column_id', 'labels', 'data', 'label_ids','column_index'])
cpa['dev'] = pd.DataFrame(val_result + val_main_cols, columns=['table_id', 'column_id', 'labels', 'data', 'label_ids','column_index'])
cpa['test'] = pd.DataFrame(test_result + test_main_cols, columns=['table_id', 'column_id', 'labels', 'data', 'label_ids','column_index'])

In [16]:
cpa['test']['column_id'] = cpa['test']['column_id'].astype(int)
cpa['train']['column_id'] = cpa['train']['column_id'].astype(int)
cpa['dev']['column_id'] = cpa['dev']['column_id'].astype(int)

In [ ]:
# Copy MLB from DODUO provided datasets
import pickle
with open('data/turl-datasets/table_rel_extraction_serialized.pkl', "rb") as f:
    train = pickle.load(f)
cpa['mlb'] = train['mlb']

In [79]:
file_name='data/sotabv2/table_rel_extraction_serialized.pkl'
f = open(file_name,'wb')
pickle.dump(cpa,f)
f.close()